In [2]:
import pandas

df_tracks = pandas.read_csv('tracks.csv',index_col=0, header=[0, 1])
df_tracks = df_tracks[df_tracks['set']['subset'] == 'medium']
df_feautures = pandas.read_csv('features.csv',index_col=0, header=[0, 1, 2])

In [3]:
df_tracks.head()

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
134             0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
136             1  2008-11-26 01:49:19  2009-01-07 00:00:00      NaN   
139             0  2008-11-26 01:49:57  2009-01-16 00:00:00      NaN   
181             0  2008-11-26 01:52:15  2007-04-13 00:00:00      NaN   

                                                                           \
         favorites  id                                        information   
track_id                                                                    
3                4   1                                            <p></p>   
134              4   1                                            <p></p>   
136              0  58  <p>A couple of unreleased mp3s from the fellas...   
139              1  60  <p>A full ensamble of strings, drums, electron...   
181              0  79  <p>This Human Ear Music reissue compiles a “Be...   

                                ...       track                         \
         listens producer tags  ... information interest language_code   
track_id                        ...                                      
3           6073      NaN   []  ...         NaN     1470            en   
134         6073      NaN   []  ...         NaN     1126            en   
136         3331      NaN   []  ...         NaN     1948            en   
139         1304      NaN   []  ...         NaN      702            en   
181         1339      NaN   []  ...         NaN     1736            en   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...     514      NaN   
134       Attribution-NonCommercial-ShareAlike 3.0 Inter...     943      NaN   
136       Attribution-NonCommercial-ShareAlike 3.0 Inter...    1498      NaN   
139       Attribution-Noncommercial-No Derivative Works ...     582      NaN   
181       Attribution-Noncommercial-No Derivative Works ...    1339      NaN   

                                                            
         number publisher tags                       title  
track_id                                                    
3             4       NaN   []                Electric Ave  
134           5       NaN   []                Street Music  
136           0       NaN   []  Peel Back The Mountain Sky  
139           2       NaN   []                    CandyAss  
181           1       NaN   []                 Gopacapulco  

[5 rows x 52 columns]

In [4]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 3 to 155314
Data columns (total 52 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (album, comments)            17000 non-null  int64  
 1   (album, date_created)        17000 non-null  object 
 2   (album, date_released)       11755 non-null  object 
 3   (album, engineer)            3351 non-null   object 
 4   (album, favorites)           17000 non-null  int64  
 5   (album, id)                  17000 non-null  int64  
 6   (album, information)         13782 non-null  object 
 7   (album, listens)             17000 non-null  int64  
 8   (album, producer)            3572 non-null   object 
 9   (album, tags)                17000 non-null  object 
 10  (album, title)               17000 non-null  object 
 11  (album, tracks)              17000 non-null  int64  
 12  (album, type)                16568 non-null  object 
 13  (artist, active_year

In [5]:
row = df_tracks.iloc[0]
row

album   comments                                                             0
        date_created                                       2008-11-26 01:44:45
        date_released                                      2009-01-05 00:00:00
        engineer                                                           NaN
        favorites                                                            4
        id                                                                   1
        information                                                    <p></p>
        listens                                                           6073
        producer                                                           NaN
        tags                                                                []
        title                                             AWOL - A Way Of Life
        tracks                                                               7
        type                                        

In [6]:
df_tracks.columns.levels[1]

Index(['active_year_begin', 'active_year_end', 'associated_labels', 'bio',
       'bit_rate', 'comments', 'composer', 'date_created', 'date_recorded',
       'date_released', 'duration', 'engineer', 'favorites', 'genre_top',
       'genres', 'genres_all', 'id', 'information', 'interest',
       'language_code', 'latitude', 'license', 'listens', 'location',
       'longitude', 'lyricist', 'members', 'name', 'number', 'producer',
       'publisher', 'related_projects', 'split', 'subset', 'tags', 'title',
       'tracks', 'type', 'website', 'wikipedia_page'],
      dtype='object')

In [7]:
print(df_tracks.index)

Index([     3,    134,    136,    139,    181,    198,    208,    209,    237,
          247,
       ...
       155292, 155293, 155294, 155295, 155296, 155297, 155298, 155306, 155307,
       155314],
      dtype='int64', name='track_id', length=17000)


In [8]:
df_tracks

album                                                            \
         comments         date_created        date_released        engineer   
track_id                                                                      
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00             NaN   
134             0  2008-11-26 01:44:45  2009-01-05 00:00:00             NaN   
136             1  2008-11-26 01:49:19  2009-01-07 00:00:00             NaN   
139             0  2008-11-26 01:49:57  2009-01-16 00:00:00             NaN   
181             0  2008-11-26 01:52:15  2007-04-13 00:00:00             NaN   
...           ...                  ...                  ...             ...   
155297          0  2017-03-30 09:50:32  2017-03-30 00:00:00             NaN   
155298          0  2017-03-30 10:45:13                  NaN             NaN   
155306          0  2017-03-30 10:45:13                  NaN             NaN   
155307          0  2017-03-30 12:51:05  2017-03-27 00:00:00  Scott Williams   
155314          0  2017-03-30 15:20:35  2017-02-17 00:00:00             NaN   

                                                                              \
         favorites     id                                        information   
track_id                                                                       
3                4      1                                            <p></p>   
134              4      1                                            <p></p>   
136              0     58  <p>A couple of unreleased mp3s from the fellas...   
139              1     60  <p>A full ensamble of strings, drums, electron...   
181              0     79  <p>This Human Ear Music reissue compiles a “Be...   
...            ...    ...                                                ...   
155297           1  22935  <p>The stars will show you the way in this unp...   
155298           0  22936                                                NaN   
155306           0  22936                                                NaN   
155307           0  22937  <p>Scott welcomes Russian revolutionary avant ...   
155314           0  22940  <p>A live performance at Monty Hall on Feb 17,...   

                                       ...       track                         \
         listens        producer tags  ... information interest language_code   
track_id                               ...                                      
3           6073             NaN   []  ...         NaN     1470            en   
134         6073             NaN   []  ...         NaN     1126            en   
136         3331             NaN   []  ...         NaN     1948            en   
139         1304             NaN   []  ...         NaN      702            en   
181         1339             NaN   []  ...         NaN     1736            en   
...          ...             ...  ...  ...         ...      ...           ...   
155297      6196             NaN   []  ...         NaN     1463           NaN   
155298      2454             NaN   []  ...         NaN      706           NaN   
155306      2454             NaN   []  ...         NaN      497           NaN   
155307      1206  Scott Williams   []  ...         NaN      630           NaN   
155314      1506     Monty Hall    []  ...         NaN      778           NaN   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...     514      NaN   
134       Attribution-NonCommercial-ShareAlike 3.0 Inter...     943      NaN   
136       Attribution-NonCommercial-ShareAlike 3.0 Inter...    1498      NaN   
139       Attribution-Noncommercial-No Derivative Works ...     582      NaN   
181       Attribution-Noncommercial-No Derivative Works ...    1339      NaN   
...      

In [9]:
# show features
df_tracks.info
df_tracks.head


<bound method NDFrame.head of             album                                                            \
         comments         date_created        date_released        engineer   
track_id                                                                      
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00             NaN   
134             0  2008-11-26 01:44:45  2009-01-05 00:00:00             NaN   
136             1  2008-11-26 01:49:19  2009-01-07 00:00:00             NaN   
139             0  2008-11-26 01:49:57  2009-01-16 00:00:00             NaN   
181             0  2008-11-26 01:52:15  2007-04-13 00:00:00             NaN   
...           ...                  ...                  ...             ...   
155297          0  2017-03-30 09:50:32  2017-03-30 00:00:00             NaN   
155298          0  2017-03-30 10:45:13                  NaN             NaN   
155306          0  2017-03-30 10:45:13                  NaN             NaN   
155307          0  201

In [21]:
import numpy as np


# filter data only for these genres
# Hip-Hop, Pop, Folk, Rock, Experimental,
# International, Electronic, and Instrumental.
# tut nix eigentlich
df_tracks = df_tracks[df_tracks['track']['genre_top'].isin(['Hip-Hop', 'Pop', 'Folk', 'Rock', 'Experimental', 'International', 'Electronic', 'Instrumental'])]




# split track dataframe into training and test data based on column with index 30
df_tracks_train = df_tracks[df_tracks.iloc[:, 30] == 'training']
df_tracks_test = df_tracks[df_tracks.iloc[:, 30] == 'test']
df_tracks_validation = df_tracks[df_tracks.iloc[:, 30] == 'validation']

#filter features data accordingly to the track data
df_features_train = df_feautures[df_feautures.index.isin(df_tracks_train.index)]
df_features_test = df_feautures[df_feautures.index.isin(df_tracks_test.index)]
df_features_validation = df_feautures[df_feautures.index.isin(df_tracks_validation.index)]

# get X
X_train = df_features_train.values
X_test = df_features_test.values
X_validation = df_features_validation.values


# get y
y_train = df_tracks_train['track']['genre_top']
y_test = df_tracks_test['track']['genre_top']
y_validation = df_tracks_validation['track']['genre_top']

# transform y to numerical values
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train)
y_test = labelencoder.fit_transform(y_test)
y_validation = labelencoder.fit_transform(y_validation)

# transform x to numerical values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_validation = scaler.fit_transform(X_validation)



# create min hashing model without using any libaries

# create a hash function
def hash_function(x, a, b, c, prime):
    return (a * x + b) % prime % c

# create a function to create a signature matrix

def create_signature_matrix(X, num_hashes):
    # create a prime number
    prime = 7841	
    # create a list to store the signature matrix
    signature_matrix = []
    # create a list to store the hash functions
    hash_functions = []
    # create n hash functions
    for i in range(num_hashes):
        a = np.random.randint(1, prime)
        b = np.random.randint(0, prime)
        hash_functions.append((a, b))
    # create signature matrix
    for i in range(num_hashes):
        signature_matrix.append([hash_function(x, hash_functions[i][0], hash_functions[i][1], len(X), prime) for x in X])
    return np.array(signature_matrix)



# test signature matrix function
signature_matrix = create_signature_matrix(X_train, 4)
print(signature_matrix.shape)














(4, 11912, 518)
(11912, 4, 518)


In [29]:
def generate_random_matrix(m, n):
    rij = np.random.choice([-1, 0, 1], size=(m, n), p=[1/6, 2/3, 1/6])
    return np.sqrt(3) * rij
# Example usage
m, n = 20, 1  # Dimensions of the matrix
random_matrix = generate_random_matrix(m, n)
print(random_matrix)

[[ 0.        ]
 [ 0.        ]
 [-1.73205081]
 [ 0.        ]
 [ 0.        ]
 [ 1.73205081]
 [ 0.        ]
 [ 1.73205081]
 [-1.73205081]
 [ 1.73205081]
 [ 0.        ]
 [-1.73205081]
 [-1.73205081]
 [ 1.73205081]
 [ 0.        ]
 [ 0.        ]
 [-1.73205081]
 [-1.73205081]
 [-1.73205081]
 [-1.73205081]]
